In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append('../')
import experiments

# for plotting across sizes ###################################################
num_net_types = 6
net_type_labels = ['Unregularized', 'All Regularizers', 'Data Augment', 'Dropout', 'Weight Decay', 'Random Labels']
net_type_colors = ['blue', 'green', 'darkviolet', 'maroon', 'orange', 'red']
net_type_styles_styles = ['-', '-', '--', '--', '--', '-']

num_sizes = 5
size_factors = [0.25, 0.5, 1, 2, 4]
size_idx_map = {0.25: 0, 0.5: 1, 1: 2, 2: 3, 4: 4}
size_labels = ['1/4x', '1/2x', '1x', '2x', '4x']

# for plotting across layers ###################################################
num_layerwise_curves = 9
layerwise_labels = ['Unregularized 1/4x', 'Unregularized 1x', 'Unregularized 4x', 'All Regularizers 1/4x',
                'All Regularizers 1x', 'All Regularizers 4x', 'Random Labels 1/4x', 'Random Labels 1x',
                'Random Labels 4x']
layerwise_colors = ['lightblue', 'blue', 'darkblue', 'lightgreen', 'limegreen', 'darkgreen', 'salmon', 'red', 'darkred']
layerwise_styles = ['-', '-', '-', '-', '-', '-', '-', '-', '-']
num_layers = 4  # 4 layers plus and 'all' layer with the averages
layer_labels = [1, 2, 3, 4]
num_crosses = 3

# for plotting robustness ###################################################

num_levels = 7
levels = np.linspace(0.0, 1.0, num=num_levels).tolist()
perturbation_range = np.linspace(0.0, 1.0, num=num_levels)
level_labels = [str(levels[i] for i in range(num_levels))]

# results ###################################################

# idx: [train/test][net_type][size][layer]
accuracy_results = np.zeros((2, num_net_types, num_sizes))

# idx: [train/test][net_type][size][layer][cross][mean/std]
similarity_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_crosses, 2))  
selectivity_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_crosses, 2))

# idx: [train/test][net_type][size][layer][cross]
compressability_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_crosses))
nonselectivity_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_crosses))

# idx: [train/test][net_type][size][layer][perturbation_level][cross]
rand_ablation_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_levels, num_crosses))
noise_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_levels, num_crosses))
targeted_ablation_results = np.zeros((2, num_net_types, num_sizes, num_layers + 1, num_levels, num_crosses))

# get data ###################################################

for ID in range(62, 92):

    print('Processing ID:', ID)
    opt = experiments.opt[ID]

    net_idx = 0  # unregularized
    if opt.dataset.random_labels:
        net_idx = 5  # random labels
    elif opt.hyper.augmentation and opt.hyper.drop_train < 1:
        net_idx = 1  # all regularizers
    elif opt.hyper.augmentation:
        net_idx = 2  # augmentation
    elif opt.hyper.drop_train < 1:
        net_idx = 3  # dropout
    elif opt.hyper.weight_decay:
        net_idx = 4  # weight decay
    size_idx = size_idx_map[opt.dnn.neuron_multiplier[0]]
    
    redundancy_file = experiments.opt[ID].csv_dir + experiments.opt[ID].name + '_redundancy.csv'
    redundancy_df = pd.read_csv(redundancy_file)
    robustness_file = experiments.opt[ID].csv_dir + experiments.opt[ID].name + '_robustness.csv'
    robustness_df = pd.read_csv(robustness_file)
        
    for eval_idx, eval_set in zip(range(2), ['train', 'test']):
        
        accuracy_results[eval_idx, net_idx, size_idx] = \
        redundancy_df[redundancy_df['evaluation_set'] == eval_set]['performance'].iloc[0]
                
        for layer_idx, layer in zip(range(num_layers + 1), list(range(num_layers)) + ['all']):
            
            for cross in range(num_crosses):
                                            
                similarity_results[eval_idx, net_idx, size_idx, layer_idx, cross, 0] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['similarity_ave'].iloc[0]
                similarity_results[eval_idx, net_idx, size_idx, layer_idx, cross, 1] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['similarity_std'].iloc[0]
                
                compressability_results[eval_idx, net_idx, size_idx, layer_idx, cross] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['compressability_95'].iloc[0]
                
                selectivity_results[eval_idx, net_idx, size_idx, layer_idx, cross, 0] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['selectivity_mean'].iloc[0]
                selectivity_results[eval_idx, net_idx, size_idx, layer_idx, cross, 1] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['selectivity_std'].iloc[0]
                
                nonselectivity_results[eval_idx, net_idx, size_idx, layer_idx, cross] = \
                redundancy_df[redundancy_df['cross_validation'] == cross] \
                [redundancy_df['evaluation_set'] == eval_set][redundancy_df['layer'] == str(layer)] \
                ['not_selective'].iloc[0]
                
                for level_idx, level in zip(range(num_levels), levels):
                    
                    rand_ablation_results[eval_idx, net_idx, size_idx, layer_idx, level_idx, cross] = \
                    robustness_df[robustness_df['cross_validation'] == cross] \
                    [robustness_df['evaluation_set'] == eval_set][robustness_df['perturbation_layer'] == str(layer)] \
                    [robustness_df['perturbation_name'] == 'Activation Knockout'] \
                    [robustness_df['perturbation_amount'] == level]['performance'].iloc[0]
                    
                    noise_results[eval_idx, net_idx, size_idx, layer_idx, level_idx, cross] = \
                    robustness_df[robustness_df['cross_validation'] == cross] \
                    [robustness_df['evaluation_set'] == eval_set][robustness_df['perturbation_layer'] == str(layer)] \
                    [robustness_df['perturbation_name'] == 'Activation Noise'] \
                    [robustness_df['perturbation_amount'] == level]['performance'].iloc[0]
                    
                    targeted_ablation_results[eval_idx, net_idx, size_idx, layer_idx, level_idx, cross] = \
                    robustness_df[robustness_df['cross_validation'] == cross] \
                    [robustness_df['evaluation_set'] == eval_set][robustness_df['perturbation_layer'] == str(layer)] \
                    [robustness_df['perturbation_name'] == 'Activation Knockout Selected'] \
                    [robustness_df['perturbation_amount'] == level]['performance'].iloc[0]
                        
accuracy_results *= 100
compressability_results *= 100
nonselectivity_results *= 100
rand_ablation_results *= 100
noise_results *= 100
targeted_ablation_results *= 100

Processing ID: 62


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:109: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:114: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:121: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/loc

Processing ID: 63
Processing ID: 64
Processing ID: 65
Processing ID: 66
Processing ID: 67
Processing ID: 68
Processing ID: 69
Processing ID: 70
Processing ID: 71
Processing ID: 72
Processing ID: 73
Processing ID: 74
Processing ID: 75
Processing ID: 76
Processing ID: 77
Processing ID: 78
Processing ID: 79
Processing ID: 80
Processing ID: 81
Processing ID: 82
Processing ID: 83
Processing ID: 84
Processing ID: 85
Processing ID: 86
Processing ID: 87
Processing ID: 88
Processing ID: 89
Processing ID: 90
Processing ID: 91


In [4]:
targeted_ablation_results

array([[[[[[100.        , 100.        , 100.        ],
           [ 84.07728041,  84.07728041,  64.97888514],
           [ 70.91638514,  70.91638514,  41.90456081],
           ...,
           [ 22.42609797,  22.42609797,  31.12964527],
           [ 19.29054054,  19.29054054,  21.61951014],
           [  9.96621622,   9.96621622,   9.96621622]],

          [[100.        , 100.        , 100.        ],
           [ 90.74746622,  89.73817568,  93.67609797],
           [ 77.08826014,  79.31376689,  83.7035473 ],
           ...,
           [ 51.46959459,  52.3120777 ,  52.24662162],
           [ 37.48310811,  34.07939189,  34.64315878],
           [ 10.00844595,  10.00844595,  10.00844595]],

          [[100.        , 100.        , 100.        ],
           [ 87.09881757,  82.52956081,  82.31418919],
           [ 60.59543919,  70.34628378,  69.45945946],
           ...,
           [ 29.54180743,  44.51224662,  47.60768581],
           [ 17.59712838,  28.83023649,  32.45565878],
           [ 